In [276]:
# load csv files in folder to map_data and fire_perim pd
import pandas as pd
import os
import glob
import numpy as np
import datetime
import matplotlib.pyplot as plt

# load FirePerimeters.csv to fire_perim pd dataframe
fire_perim = pd.read_csv('/Users/divya/Desktop/Project4/FirePerimeters_old.csv')

fire_perim

,OBJECTID,Year,State,Agency,Unit ID,Fire Name,Local Incident Number,Alarm Date,Containment Date,Cause,...,Management Objective,GIS Calculated Acres,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,DECADES,Shape__Area,Shape__Length
0,1,2023.0,CA,CDF,SKU,WHITWORTH,00004808,6/17/2023 12:00:00 AM,6/17/2023 12:00:00 AM,5.0,...,1.0,5.729125,NaN,NaN,{7985848C-0AC2-4BA4-8F0E-29F778652E61},NaN,NaN,2020.0,4.140784e+04,1247.166034
1,2,2023.0,CA,LRA,BTU,KAISER,00010225,6/2/2023 12:00:00 AM,6/2/2023 12:00:00 AM,5.0,...,1.0,13.602380,NaN,NaN,{43EBCC88-B3AC-48EB-8EF5-417FE0939CCF},NaN,NaN,2020.0,9.345588e+04,1285.514550
2,3,2023.0,CA,CDF,AEU,JACKSON,00017640,7/1/2023 12:00:00 AM,7/2/2023 12:00:00 AM,2.0,...,1.0,27.814460,NaN,NaN,{B64E1355-BF1D-441A-95D0-BC1FBB93483B},NaN,NaN,2020.0,1.830285e+05,2697.587429
3,4,2023.0,CA,CDF,AEU,CARBON,00018821,7/11/2023 12:00:00 AM,7/11/2023 12:00:00 AM,9.0,...,1.0,58.760230,NaN,NaN,{CB41DB0A-E4B1-489D-A4EA-738F2CD6DB3B},NaN,NaN,2020.0,3.882220e+05,2548.738779
4,5,2023.0,CA,CDF,AEU,LIBERTY,00018876,7/11/2023 12:00:00 AM,7/12/2023 12:00:00 AM,14.0,...,1.0,70.979000,NaN,NaN,{F83F70A4-07A7-40B8-BD51-10CCC1C30D63},NaN,NaN,2020.0,4.664560e+05,5106.936526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22256,22257,NaN,CA,CCO,MRN,UKNOWN,NaN,NaN,NaN,14.0,...,1.0,1494.277000,1917-34(Yr Not Report)MarinCo FireChief Garber...,NaN,NaN,NaN,NaN,NaN,9.768233e+06,11458.048312
22257,22258,NaN,CA,CCO,MRN,UKNOWN,NaN,NaN,NaN,14.0,...,1.0,465.534600,1917-34(Yr Not Report)MarinCo FireChief Garber...,NaN,NaN,NaN,NaN,NaN,3.048457e+06,6265.090923
22258,22259,NaN,CA,CCO,MRN,UKNOWN,NaN,NaN,NaN,14.0,...,1.0,2927.242000,1917-34(Yr Not Report)MarinCo FireChief Garber...,NaN,NaN,NaN,NaN,NaN,1.919632e+07,18325.306274
22259,22260,NaN,CA,CCO,MRN,UKNOWN,NaN,NaN,NaN,14.0,...,1.0,62.012680,1917-34(Yr Not Report)MarinCo FireChief Garber...,NaN,NaN,NaN,NaN,NaN,4.048420e+05,2344.122215


In [277]:
print(fire_perim.columns)

Index(['OBJECTID', 'Year', 'State', 'Agency', 'Unit ID', 'Fire Name',
       'Local Incident Number', 'Alarm Date', 'Containment Date', 'Cause',
       'Collection Method', 'Management Objective', 'GIS Calculated Acres',
       'Comments', 'Complex Name', 'IRWIN ID', 'Fire Number (historical use)',
       'Complex ID', 'DECADES', 'Shape__Area', 'Shape__Length'],
      dtype='object')


In [278]:
#in fire_perim: drop the following columns: Comments, Complex Name, Decades, Fire Number (historical use), Complex ID
fire_perim = fire_perim.drop(columns=['Comments', 'Complex Name', 'DECADES', 'Fire Number (historical use)', 'Complex ID'])

#drop rows where state is not equal to CA
fire_perim = fire_perim[fire_perim['State'] == 'CA']
fire_perim = fire_perim.reset_index(drop=True)
fire_perim = fire_perim.drop(columns=['State'])


In [279]:
fire_perim.columns = fire_perim.columns.str.lower()
fire_perim.columns = fire_perim.columns.str.replace(' ', '_')
fire_perim.columns = fire_perim.columns.str.replace('__', '_')

fire_perim['year'] = fire_perim['year'].fillna(0).astype(int)
fire_perim['cause'] = fire_perim['cause'].fillna(0).astype(int)
fire_perim['collection_method'] = fire_perim['collection_method'].fillna(0).astype(int)
fire_perim['management_objective'] = fire_perim['management_objective'].fillna(0).astype(int)

In [280]:
fire_perim['alarm_date'] = fire_perim['alarm_date'].str.replace('/0219', '/2019')
fire_perim['containment_date'] = fire_perim['containment_date'].str.replace('/0219', '/2019')

fire_perim['alarm_date'] = pd.to_datetime(fire_perim['alarm_date'], format='%m/%d/%Y %I:%M:%S %p')
fire_perim['containment_date'] = pd.to_datetime(fire_perim['containment_date'], format='%m/%d/%Y %I:%M:%S %p')

#turn unit_id, fire_name, local_incident_number, and irwin_id to str
fire_perim['unit_id'] = fire_perim['unit_id'].astype(str)
fire_perim['fire_name'] = fire_perim['fire_name'].astype(str)
fire_perim['local_incident_number'] = fire_perim['local_incident_number'].astype(str)
fire_perim['irwin_id'] = fire_perim['irwin_id'].astype(str)


In [281]:
fire_perim.dtypes

objectid                          int64
year                              int64
agency                           object
unit_id                          object
fire_name                        object
local_incident_number            object
alarm_date               datetime64[ns]
containment_date         datetime64[ns]
cause                             int64
collection_method                 int64
management_objective              int64
gis_calculated_acres            float64
irwin_id                         object
shape_area                      float64
shape_length                    float64
dtype: object

In [282]:
#drop rows in which year = 0 
fire_perim = fire_perim[fire_perim['year'] != 0]
fire_perim

,objectid,year,agency,unit_id,fire_name,local_incident_number,alarm_date,containment_date,cause,collection_method,management_objective,gis_calculated_acres,irwin_id,shape_area,shape_length
0,1,2023,CDF,SKU,WHITWORTH,00004808,2023-06-17,2023-06-17,5,1,1,5.729125,{7985848C-0AC2-4BA4-8F0E-29F778652E61},4.140784e+04,1247.166034
1,2,2023,LRA,BTU,KAISER,00010225,2023-06-02,2023-06-02,5,1,1,13.602380,{43EBCC88-B3AC-48EB-8EF5-417FE0939CCF},9.345588e+04,1285.514550
2,3,2023,CDF,AEU,JACKSON,00017640,2023-07-01,2023-07-02,2,1,1,27.814460,{B64E1355-BF1D-441A-95D0-BC1FBB93483B},1.830285e+05,2697.587429
3,4,2023,CDF,AEU,CARBON,00018821,2023-07-11,2023-07-11,9,1,1,58.760230,{CB41DB0A-E4B1-489D-A4EA-738F2CD6DB3B},3.882220e+05,2548.738779
4,5,2023,CDF,AEU,LIBERTY,00018876,2023-07-11,2023-07-12,14,1,1,70.979000,{F83F70A4-07A7-40B8-BD51-10CCC1C30D63},4.664560e+05,5106.936526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22163,22180,1898,CCO,VNC,MATILIJA,00000000,1898-04-01,NaT,14,0,1,2641.123000,nan,1.577012e+07,27417.390907
22164,22181,1898,CCO,VNC,COZY DELL,00000000,1898-09-01,NaT,14,0,1,2974.585000,nan,1.775922e+07,18817.257350
22165,22182,1896,CCO,LAC,,00000000,NaT,NaT,14,0,1,7531.717000,nan,4.473243e+07,36532.907435
22166,22183,1895,CCO,LAC,,00000000,NaT,NaT,14,0,1,628.113600,nan,3.727269e+06,7182.120280


In [283]:
#print column names
print(fire_perim.columns)


Index(['objectid', 'year', 'agency', 'unit_id', 'fire_name',
       'local_incident_number', 'alarm_date', 'containment_date', 'cause',
       'collection_method', 'management_objective', 'gis_calculated_acres',
       'irwin_id', 'shape_area', 'shape_length'],
      dtype='object')


In [284]:
#create new column called days_to_contain in which alarm date is subtracted from containment date to find the amount of days it took to contain
fire_perim['days_to_contain'] = fire_perim['containment_date'] - fire_perim['alarm_date']

In [285]:
fire_perim

,objectid,year,agency,unit_id,fire_name,local_incident_number,alarm_date,containment_date,cause,collection_method,management_objective,gis_calculated_acres,irwin_id,shape_area,shape_length,days_to_contain
0,1,2023,CDF,SKU,WHITWORTH,00004808,2023-06-17,2023-06-17,5,1,1,5.729125,{7985848C-0AC2-4BA4-8F0E-29F778652E61},4.140784e+04,1247.166034,0 days
1,2,2023,LRA,BTU,KAISER,00010225,2023-06-02,2023-06-02,5,1,1,13.602380,{43EBCC88-B3AC-48EB-8EF5-417FE0939CCF},9.345588e+04,1285.514550,0 days
2,3,2023,CDF,AEU,JACKSON,00017640,2023-07-01,2023-07-02,2,1,1,27.814460,{B64E1355-BF1D-441A-95D0-BC1FBB93483B},1.830285e+05,2697.587429,1 days
3,4,2023,CDF,AEU,CARBON,00018821,2023-07-11,2023-07-11,9,1,1,58.760230,{CB41DB0A-E4B1-489D-A4EA-738F2CD6DB3B},3.882220e+05,2548.738779,0 days
4,5,2023,CDF,AEU,LIBERTY,00018876,2023-07-11,2023-07-12,14,1,1,70.979000,{F83F70A4-07A7-40B8-BD51-10CCC1C30D63},4.664560e+05,5106.936526,1 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22163,22180,1898,CCO,VNC,MATILIJA,00000000,1898-04-01,NaT,14,0,1,2641.123000,nan,1.577012e+07,27417.390907,NaT
22164,22181,1898,CCO,VNC,COZY DELL,00000000,1898-09-01,NaT,14,0,1,2974.585000,nan,1.775922e+07,18817.257350,NaT
22165,22182,1896,CCO,LAC,,00000000,NaT,NaT,14,0,1,7531.717000,nan,4.473243e+07,36532.907435,NaT
22166,22183,1895,CCO,LAC,,00000000,NaT,NaT,14,0,1,628.113600,nan,3.727269e+06,7182.120280,NaT


In [286]:
#export fire_perim to test.csv
fire_perim.to_csv('/Users/divya/Desktop/Project4/fire_perim_final.csv', index=False)

In [287]:
map_data = pd.read_csv('/Users/divya/Desktop/Project4/cdf_firedata.csv')

In [288]:
map_data

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_latitude,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,40.774000,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False
1,Pala Fire,Y,2020-09-16T14:07:35Z,2009-05-24T14:56:00Z,CAL FIRE San Diego Unit,NaN,San Diego,"Hwy 76 and Pala Temecula, northwest of Pala",122.0,100.0,...,1.000000,Wildfire,8f61f461-552d-4538-b186-35ab030da416,https://www.fire.ca.gov/incidents/2009/5/24/pa...,2009-05-25T00:00:00Z,2009-05-25,2009-05-24,N,True,False
2,River Fire,Y,2022-10-24T11:39:23Z,2013-02-24T08:16:00Z,CAL FIRE San Bernardino Unit,NaN,Inyo,"south of Narrow Gauge Rd & north of Hwy 136, e...",407.0,100.0,...,36.602575,NaN,094719ba-a47b-4abb-9ec5-a506b2b9fd23,https://www.fire.ca.gov/incidents/2013/2/24/ri...,2013-02-28T20:00:00Z,2013-02-28,2013-02-24,N,True,False
3,Fawnskin Fire,Y,2013-04-22T09:00:00Z,2013-04-20T17:30:00Z,San Bernardino National Forest,NaN,San Bernardino,"west of Delamar Mountain, north of the communi...",30.0,100.0,...,34.288877,NaN,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,https://www.fire.ca.gov/incidents/2013/4/20/fa...,2013-04-22T09:00:00Z,2013-04-22,2013-04-20,N,False,False
4,Gold Fire,Y,2013-05-01T07:00:00Z,2013-04-30T12:59:00Z,CAL FIRE Madera-Mariposa-Merced Unit,NaN,Madera,Between Road 210 and Road 200 near Fine Gold C...,274.0,100.0,...,37.116295,NaN,357ffc13-bef9-48eb-810f-c5de851972eb,https://www.fire.ca.gov/incidents/2013/4/30/go...,2013-05-01T07:00:00Z,2013-05-01,2013-04-30,N,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,San Joaquin Fire,Y,2024-07-16T10:54:49Z,2024-07-15T22:03:34Z,CAL FIRE Sonoma-Lake-Napa Unit,NaN,Lake,Mendocino Street and San Joaquin Ave in Clearl...,8.0,100.0,...,38.979194,Wildfire,c9806a95-09ca-4b22-97ae-a9f822608e6d,https://www.fire.ca.gov/incidents/2024/7/15/sa...,NaN,NaN,2024-07-15,N,False,False
2512,Hill Fire,N,2024-07-16T23:00:22Z,2024-07-16T11:05:51Z,Six Rivers National Forest,NaN,Humboldt,South of Willow Creek in the Mosquito Creek Dr...,3437.0,NaN,...,40.772161,Wildfire,915d22c2-22e8-4720-a486-196e23f31150,https://www.fire.ca.gov/incidents/2024/7/16/hi...,NaN,NaN,2024-07-16,Y,False,False
2513,Long Fire,N,2024-07-16T18:08:05Z,2024-07-16T15:32:00Z,CAL FIRE Tulare Unit,NaN,Tulare,5 miles East of Black Mountain in Domeland Wil...,160.0,0.0,...,35.827215,Wildfire,42cc2477-9e8b-429b-9dcb-b2786dc4be42,https://www.fire.ca.gov/incidents/2024/7/16/lo...,NaN,NaN,2024-07-16,Y,True,False
2514,Willow Fire,N,2024-07-16T17:31:08Z,2024-07-16T17:04:46Z,Kern County Fire Department,NaN,Kern,"Near Backus Road and 85th Street, Willow Springs",175.0,NaN,...,34.922210,Wildfire,f27ffd72-b167-4212-a4f9-a8f8099469e0,https://www.fire.ca.gov/incidents/2024/7/16/wi...,NaN,NaN,2024-07-16,Y,False,False


In [289]:
map_data.columns

Index(['incident_name', 'incident_is_final', 'incident_date_last_update',
       'incident_date_created', 'incident_administrative_unit',
       'incident_administrative_unit_url', 'incident_county',
       'incident_location', 'incident_acres_burned', 'incident_containment',
       'incident_control', 'incident_cooperating_agencies',
       'incident_longitude', 'incident_latitude', 'incident_type',
       'incident_id', 'incident_url', 'incident_date_extinguished',
       'incident_dateonly_extinguished', 'incident_dateonly_created',
       'is_active', 'calfire_incident', 'notification_desired'],
      dtype='object')

In [290]:
map_data.head(1)

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_administrative_unit_url,incident_county,incident_location,incident_acres_burned,incident_containment,...,incident_latitude,incident_type,incident_id,incident_url,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active,calfire_incident,notification_desired
0,Bridge Fire,Y,2018-01-09T13:46:00Z,2017-10-31T11:22:00Z,Shasta-Trinity National Forest,NaN,Shasta,"I-5 and Turntable Bay, 7 miles NE of Shasta Lake",37.0,100.0,...,40.774,NaN,2ca11d45-8139-4c16-8af0-880d99b21e82,https://www.fire.ca.gov/incidents/2017/10/31/b...,2018-01-09T13:46:00Z,2018-01-09,2017-10-31,N,False,False


In [291]:
map_data.dtypes

incident_name                        object
incident_is_final                    object
incident_date_last_update            object
incident_date_created                object
incident_administrative_unit         object
incident_administrative_unit_url    float64
incident_county                      object
incident_location                    object
incident_acres_burned               float64
incident_containment                float64
incident_control                     object
incident_cooperating_agencies        object
incident_longitude                  float64
incident_latitude                   float64
incident_type                        object
incident_id                          object
incident_url                         object
incident_date_extinguished           object
incident_dateonly_extinguished       object
incident_dateonly_created            object
is_active                            object
calfire_incident                       bool
notification_desired            

In [292]:
# Convert object columns to string
map_data['incident_name'] = map_data['incident_name'].astype(str)
map_data['incident_is_final'] = map_data['incident_is_final'].astype(str)
map_data['incident_administrative_unit'] = map_data['incident_administrative_unit'].astype(str)
map_data['incident_county'] = map_data['incident_county'].astype(str)
map_data['incident_location'] = map_data['incident_location'].astype(str)
map_data['incident_control'] = map_data['incident_control'].astype(str)
map_data['incident_cooperating_agencies'] = map_data['incident_cooperating_agencies'].astype(str)
map_data['incident_type'] = map_data['incident_type'].astype(str)
map_data['incident_id'] = map_data['incident_id'].astype(str)
map_data['is_active'] = map_data['is_active'].astype(str)

# Convert object columns to datetime
map_data['incident_date_last_update'] = pd.to_datetime(map_data['incident_date_last_update'])
map_data['incident_date_created'] = pd.to_datetime(map_data['incident_date_created'])
map_data['incident_date_extinguished'] = pd.to_datetime(map_data['incident_date_extinguished'])
map_data['incident_dateonly_extinguished'] = pd.to_datetime(map_data['incident_dateonly_extinguished'])
map_data['incident_dateonly_created'] = pd.to_datetime(map_data['incident_dateonly_created'])

# Drop the URL columns
map_data = map_data.drop(columns=['incident_administrative_unit_url', 'incident_url','incident_location', 'notification_desired', 'calfire_incident'])

# Verify the data types
print(map_data.dtypes)

incident_name                                  object
incident_is_final                              object
incident_date_last_update         datetime64[ns, UTC]
incident_date_created             datetime64[ns, UTC]
incident_administrative_unit                   object
incident_county                                object
incident_acres_burned                         float64
incident_containment                          float64
incident_control                               object
incident_cooperating_agencies                  object
incident_longitude                            float64
incident_latitude                             float64
incident_type                                  object
incident_id                                    object
incident_date_extinguished        datetime64[ns, UTC]
incident_dateonly_extinguished         datetime64[ns]
incident_dateonly_created              datetime64[ns]
is_active                                      object
dtype: object


In [293]:
map_data

,incident_name,incident_is_final,incident_date_last_update,incident_date_created,incident_administrative_unit,incident_county,incident_acres_burned,incident_containment,incident_control,incident_cooperating_agencies,incident_longitude,incident_latitude,incident_type,incident_id,incident_date_extinguished,incident_dateonly_extinguished,incident_dateonly_created,is_active
0,Bridge Fire,Y,2018-01-09 13:46:00+00:00,2017-10-31 11:22:00+00:00,Shasta-Trinity National Forest,Shasta,37.0,100.0,nan,Shasta-Trinity National Forest,-122.309000,40.774000,nan,2ca11d45-8139-4c16-8af0-880d99b21e82,2018-01-09 13:46:00+00:00,2018-01-09,2017-10-31,N
1,Pala Fire,Y,2020-09-16 14:07:35+00:00,2009-05-24 14:56:00+00:00,CAL FIRE San Diego Unit,San Diego,122.0,100.0,nan,CAL FIRE San Diego Unit,1.000000,1.000000,Wildfire,8f61f461-552d-4538-b186-35ab030da416,2009-05-25 00:00:00+00:00,2009-05-25,2009-05-24,N
2,River Fire,Y,2022-10-24 11:39:23+00:00,2013-02-24 08:16:00+00:00,CAL FIRE San Bernardino Unit,Inyo,407.0,100.0,nan,"CAL FIRE San Bernardino Unit, Inyo County Sher...",-118.016510,36.602575,nan,094719ba-a47b-4abb-9ec5-a506b2b9fd23,2013-02-28 20:00:00+00:00,2013-02-28,2013-02-24,N
3,Fawnskin Fire,Y,2013-04-22 09:00:00+00:00,2013-04-20 17:30:00+00:00,San Bernardino National Forest,San Bernardino,30.0,100.0,nan,San Bernardino National Forest,-116.941311,34.288877,nan,58f89ff8-bd3e-4355-b1c0-8fa05c747d3f,2013-04-22 09:00:00+00:00,2013-04-22,2013-04-20,N
4,Gold Fire,Y,2013-05-01 07:00:00+00:00,2013-04-30 12:59:00+00:00,CAL FIRE Madera-Mariposa-Merced Unit,Madera,274.0,100.0,nan,CAL FIRE Madera-Mariposa-Merced Unit,-119.635004,37.116295,nan,357ffc13-bef9-48eb-810f-c5de851972eb,2013-05-01 07:00:00+00:00,2013-05-01,2013-04-30,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2511,San Joaquin Fire,Y,2024-07-16 10:54:49+00:00,2024-07-15 22:03:34+00:00,CAL FIRE Sonoma-Lake-Napa Unit,Lake,8.0,100.0,nan,nan,-122.684661,38.979194,Wildfire,c9806a95-09ca-4b22-97ae-a9f822608e6d,NaT,NaT,2024-07-15,N
2512,Hill Fire,N,2024-07-16 23:00:22+00:00,2024-07-16 11:05:51+00:00,Six Rivers National Forest,Humboldt,3437.0,NaN,nan,nan,-123.639751,40.772161,Wildfire,915d22c2-22e8-4720-a486-196e23f31150,NaT,NaT,2024-07-16,Y
2513,Long Fire,N,2024-07-16 18:08:05+00:00,2024-07-16 15:32:00+00:00,CAL FIRE Tulare Unit,Tulare,160.0,0.0,nan,nan,-118.175425,35.827215,Wildfire,42cc2477-9e8b-429b-9dcb-b2786dc4be42,NaT,NaT,2024-07-16,Y
2514,Willow Fire,N,2024-07-16 17:31:08+00:00,2024-07-16 17:04:46+00:00,Kern County Fire Department,Kern,175.0,NaN,nan,nan,-118.257558,34.922210,Wildfire,f27ffd72-b167-4212-a4f9-a8f8099469e0,NaT,NaT,2024-07-16,Y


In [294]:
map_data.to_csv('/Users/divya/Desktop/Project4/map_data_final.csv',index=False)